<a href="https://colab.research.google.com/github/jihyeon602/DeepLearning_practice/blob/main/%EA%B5%B0%EA%B2%BD%EA%B3%84%EC%9E%91%EC%A0%84%ED%99%98%EA%B2%BD%ED%95%A9%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 군 경계 작전 환경 합성 데이터
- 문제정의: **여러 class를 실시간 탐지하는 모델 생성 및 추론 테스트**

- **dataset**: EO(RGB)_SU(하절기)_DT(주간) 데이터셋만 사용  #용량이슈
  - 원본 그대로 TRAIN은 train val로 구성

- **annotations**: 문서에 있는 class만 실시간 탐지하도록 구성
  - GPU 사용 제한 시 샘플링❗

- **산출물**: model파일(.pt 등), results

### 데이터 로드

In [ ]:
import zipfile

zip_file_path = './3.개방데이터/1.데이터/Training/02.라벨링데이터/TL_EO_SU_DT.zip'
extract_to = './AMY/labels_json/train'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("압축 해제 완료!")

In [ ]:
import zipfile

zip_file_path = './3.개방데이터/1.데이터/Training/01.원천데이터/TS_EO_SU_DT.zip'
extract_to = './AMY/images/train'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("압축 해제 완료!")

In [ ]:
import zipfile

zip_file_path = './3.개방데이터/1.데이터/Validation/02.라벨링데이터/VL_EO_SU_DT.zip'
extract_to = './AMY/labels_json/val'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("압축 해제 완료!")

In [ ]:
import zipfile

zip_file_path = './3.개방데이터/1.데이터/Validation/01.원천데이터/VS_EO_SU_DT.zip'
extract_to = './AMY/images/val'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("압축 해제 완료!")

### json 파일 yolo 포맷(.txt) 파일로 변환

In [ ]:
import os
import json
from PIL import Image

# 기본 경로
base_path = "./3.개방데이터/1.데이터"

# 경로 딕셔너리
sets = {
    'train': {
        'json_folder': os.path.join(base_path, "labels_json", "train"),   # JSON 라벨
        'image_folder': os.path.join(base_path, "images", "train"),  # 이미지
        'output_folder': os.path.join(base_path, "labels", "train")  # YOLO 형식 저장될 위치
    },
    'val': {
        'json_folder': os.path.join(base_path, "labels_json", "val"),
        'image_folder': os.path.join(base_path, "images", "val"),
        'output_folder': os.path.join(base_path, "labels", "val")
    }
}

# 지원 이미지 확장자
image_exts = ['.jpg', '.jpeg', '.png']

# 변환 함수 정의
def convert_to_yolo(json_folder, image_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for file in os.listdir(json_folder):
        if file.endswith('.json'):
            base_name = os.path.splitext(file)[0]

            # JSON 로드
            with open(os.path.join(json_folder, file), 'r', encoding='utf-8') as f:
                data = json.load(f)

            # 이미지 찾기
            image_path = None
            for ext in image_exts:
                candidate = os.path.join(image_folder, base_name + ext)
                if os.path.exists(candidate):
                    image_path = candidate
                    break

            if not image_path:
                print(f"⚠️ 이미지 없음: {base_name}")
                continue

            # 이미지 크기
            with Image.open(image_path) as img:
                img_width, img_height = img.size

            yolo_lines = []
            for ann in data.get("annotations", []):
                if ann.get("shape") == "Bounding Box":
                    (x1, y1), (x2, y2) = ann["points"]
                    cx = (x1 + x2) / 2 / img_width
                    cy = (y1 + y2) / 2 / img_height
                    w = abs(x2 - x1) / img_width
                    h = abs(y2 - y1) / img_height
                    yolo_lines.append(f"0 {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}")

            # 결과 저장
            out_path = os.path.join(output_folder, base_name + ".txt")
            with open(out_path, "w", encoding='utf-8') as f:
                f.write("\n".join(yolo_lines))

            print(f"✅ 변환 완료: {base_name}.txt")

# ▶ 각 세트(train, val)에 대해 실행
for phase, paths in sets.items():
    print(f"\n📂 처리 중: {phase}")
    convert_to_yolo(paths['json_folder'], paths['image_folder'], paths['output_folder'])
